# 준비

먼저 Naver Cloud Platform에서 지오코딩 API를 사용할 수 있도록 API Key를 받습니다.  
https://api.ncloud-docs.com/docs/ko/ai-naver-mapsgeocoding 

In [1]:
# 데이터 처리
import numpy as np
import pandas as pd

In [2]:
# "전국CCTV표준데이터" CSV 파일 로딩 
df_cctv = pd.read_csv('./data/전국CCTV표준데이터.csv', encoding="EUC-KR", header=0, 
                           names=['admin_nm', 'addr_rd', 'addr_jb', 'purpose', 'cctv_cnt', 'pixel', 'direction', 'period', 'date_install', 'tel', 'lat', 'lon', 'date_make', 'offer_admin_cd', 'offer_admin_nm'], 
                           dtype={'cctv_cnt':object, 'pixel':object, 'period':object, 'lat':float, 'lon':float, 'offer_admin_cd':object}, 
                  thousands = ',', #천단위 쉼표 제거   
                  index_col=None) 

In [4]:
df_cctv.dropna(subset=['lat'],axis=0,inplace=True)  #거리 비교를 위해 null 삭제
df_cctv.dropna(subset=['lon'],axis=0,inplace=True)  #거리 비교를 위해 null 삭제
df_cctv.dropna(subset=['addr_rd'],axis=0,inplace=True) # 도로명주소 nulll 데이터 삭제
df_cctv.dropna(subset=['addr_jb'],axis=0,inplace=True) #지번주소 null 데이터 삭제 

In [5]:
# 5만 건 중 0.01% 샘플링
df_cctv2 = df_cctv.sample(frac=0.001)
df_cctv2.info()
df_cctv2.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24 entries, 30327 to 29145
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   admin_nm        24 non-null     object 
 1   addr_rd         24 non-null     object 
 2   addr_jb         24 non-null     object 
 3   purpose         24 non-null     object 
 4   cctv_cnt        24 non-null     object 
 5   pixel           23 non-null     object 
 6   direction       22 non-null     object 
 7   period          24 non-null     object 
 8   date_install    21 non-null     object 
 9   tel             24 non-null     object 
 10  lat             24 non-null     float64
 11  lon             24 non-null     float64
 12  date_make       24 non-null     object 
 13  offer_admin_cd  24 non-null     object 
 14  offer_admin_nm  24 non-null     object 
dtypes: float64(2), object(13)
memory usage: 3.0+ KB


,admin_nm,addr_rd,addr_jb,purpose,cctv_cnt,pixel,direction,period,date_install,tel,lat,lon,date_make,offer_admin_cd,offer_admin_nm
30327,대구광역시 서구청,대구광역시 서구 새방로25길 46,대구광역시 서구 상리동 644,다목적,1,200,남,30,2011-12,053-663-3901,35.866413,128.525622,2020-12-24,3430000,대구광역시 서구
40686,강원도 원주시청,소방서길 22-5,명륜2동 820-6,생활방범,1,200,360,30,NaN,033-737-5153,37.329761,127.944557,2020-09-20,4190000,강원도 원주시
37776,공주시청,충청남도 공주시 금벽로 368,충청남도 공주시 신관동 553천(금강신관공원공주대교 아래),생활방범,1,130,360도전방면,30,2016-12,041-840-8500,36.462876,127.127437,2020-03-25,4500000,충청남도 공주시
26562,부산광역시 금정구청,부산광역시 금정구 동부곡로15번길 3,부산광역시 금정구 부곡동 287-6,생활방범,1,200,360도 전방면,30,2020-05,051-519-5731,35.225864,129.093181,2020-08-12,3350000,부산광역시 금정구
19917,경상남도 창원시,경상남도 창원시 의창구 창이대로397번길 2,경상남도 창원시 의창구 사림동 75-1,교통정보수집,4,130,360도 전방면,30,2011-06,055-225-3111,35.242717,128.680032,2020-08-25,5670000,경상남도 창원시


# Geopy - Nominatim을 이용한 지오코딩  
### Geopy  
- 지오코딩 패키지 (https://geopy.readthedocs.io/en/stable/#  )       
- Geopy는 Nominatim(https://geopy.readthedocs.io/en/stable/#nominatim) 뿐만 아니라   
  Google 등 다양한 지오코딩 서비스를 지원함 (다만 한국 포털 지도 API는 아직 없는 것으로 보임)   

### Nominatim   
- OpenStreetMap의 지명이나 주소를 이용하여 지오코딩을 할 수 있는 서비스 (라틴어로서, '이름'이라는 뜻)  
 : https://nominatim.org/  
 : https://wiki.openstreetmap.org/wiki/Ko:Nominatim  
 : https://github.com/osm-search/Nominatim 
 
### Geopy를 이용한 Nominatim 사용 예제  
 https://towardsdatascience.com/things-to-do-with-latitude-longitude-data-using-geopy-python-1d356ed1ae30

In [7]:
#import geopy  #!pip install geopy
import ssl
import urllib3
import requests
from geopy.geocoders import Nominatim
from  geopy.extra.rate_limiter import RateLimiter 
geolocator = Nominatim(user_agent='testing2')  #South Korea
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=2, return_value_on_exception = None) 

In [8]:
# 아래 3가지 코드 둥의 하나를 실행 
df_cctv2['location'] = df_cctv2['addr_rd'].apply(geocode)
df_cctv2['lat2'] = df_cctv2['location'].apply(lambda x: x.latitude if x else None)
df_cctv2['lon2'] = df_cctv2['location'].apply(lambda x: x.longitude if x else None)

In [9]:
df_cctv2

,admin_nm,addr_rd,addr_jb,purpose,cctv_cnt,pixel,direction,period,date_install,tel,lat,lon,date_make,offer_admin_cd,offer_admin_nm,location,lat2,lon2
30327,대구광역시 서구청,대구광역시 서구 새방로25길 46,대구광역시 서구 상리동 644,다목적,1,200,남,30,2011-12,053-663-3901,35.866413,128.525622,2020-12-24,3430000,대구광역시 서구,None,NaN,NaN
40686,강원도 원주시청,소방서길 22-5,명륜2동 820-6,생활방범,1,200,360,30,NaN,033-737-5153,37.329761,127.944557,2020-09-20,4190000,강원도 원주시,"(소방서길, 명륜동, 원주시, 강원도, 26481, 대한민국, (37.3316091...",37.331609,127.944570
37776,공주시청,충청남도 공주시 금벽로 368,충청남도 공주시 신관동 553천(금강신관공원공주대교 아래),생활방범,1,130,360도전방면,30,2016-12,041-840-8500,36.462876,127.127437,2020-03-25,4500000,충청남도 공주시,"(금벽로, 노은1동, 노은동, 온천리, 공주시, 충청남도, 34158, 대한민국, ...",36.363186,127.268690
26562,부산광역시 금정구청,부산광역시 금정구 동부곡로15번길 3,부산광역시 금정구 부곡동 287-6,생활방범,1,200,360도 전방면,30,2020-05,051-519-5731,35.225864,129.093181,2020-08-12,3350000,부산광역시 금정구,"(동부곡로15번길, 부곡2동, 부곡동, 금정구, 부산, 46271, 대한민국, (3...",35.228317,129.093780
19917,경상남도 창원시,경상남도 창원시 의창구 창이대로397번길 2,경상남도 창원시 의창구 사림동 75-1,교통정보수집,4,130,360도 전방면,30,2011-06,055-225-3111,35.242717,128.680032,2020-08-25,5670000,경상남도 창원시,"(창이대로397번길, 의창구, 창원시, 경상남도, 51431, 대한민국, (35.2...",35.244254,128.680770
7980,서울특별시 노원구청,서울특별시 노원구 중계로12가길29,서울특별시 노원구 중계동 69,생활방범,1,200,회전,30,2015-12,02-2116-4917,37.647620,127.082171,2019-04-08,3100000,서울특별시 노원구,None,NaN,NaN
45271,대전광역시청,대전광역시 중구 동서대로 1365번길 13,대전광역시 중구 목동 96-2,기타,1,41,문화재방면,30,2010-02,042-270-4524,36.331998,127.408657,2017-02-01,6300000,대전광역시,None,NaN,NaN
31971,전라북도 군산시,전라북도 군산시 번영로 2,전라북도 군산시 경장동 472,생활방범,2,200,360도 전방면,30,2017-12,063-454-7922,35.975177,126.723794,2020-03-12,4670000,전라북도 군산시,"(번영로, 대야면, 군산시, 전라북도, 54060, 대한민국, (35.9413032...",35.941303,126.831394
13635,경기도 시흥시청,경기도 시흥시 정왕천로375번길 52-1,경기도 시흥시 정왕동 1571-3,생활방범,4,200,360도전방면,30,2019-06,031-310-6752,37.344315,126.746445,2020-04-14,4010000,경기도 시흥시,None,NaN,NaN
20776,경기도 이천시청,경기도 이천시 장호원읍 서동대로8830번길 3,경기도 이천시 장호원읍 진암리 산 28-12,생활방범,1,200,120,30,2018-05,031-644-2942,37.115154,127.616365,2020-01-20,4070000,경기도 이천시,None,NaN,NaN


In [10]:
# df_cctv2['temp'] = df_cctv2['addr_rd'].apply(lambda loc: tuple(loc.point) if loc else None)
# df_cctv2.head()

In [8]:
# df_cctv2['temp'] = df_cctv2['addr_rd'].apply(geocode) 
# df_cctv2.head()

,admin_nm,addr_rd,addr_jb,purpose,cctv_cnt,pixel,direction,period,date_install,tel,lat,lon,date_make,offer_admin_cd,offer_admin_nm,coords
15745,대구광역시 중구청,대구광역시 중구 남산로7길 75,대구광역시 중구 남산동 2482-458,어린이보호,1,200,360도 전방면,30,2016-05,053-661-2494,35.859354,128.579142,2018-06-30,3410000,대구광역시 중구,"(남산로7길, 까치아파트, 남산동, 중구, 대구, 41977, 대한민국, (35.8..."
34151,서울특별시 동대문구,서울특별시 동대문구 천호대로85길 56,서울특별시 동대문구 장안1동 447-7,생활방범,3,200,NaN,NaN,2007-01,02-2127-4097,37.564096,127.067368,2020-10-23,3050000,서울특별시 동대문구,"(천호대로85길, Jangan 1(il)-dong, 동대문구, 서울, 노원구, 02..."
3492,경상남도 진주시청,경상남도 진주시 충무공동 바람모아생활방범 전망데크,경상남도 진주시 충무공동 바람모아생활방범 전망데크,생활방범,4,200,전방 50미터,30,2018-12,055-749-5566,35.175577,128.132953,2019-10-23,5310000,경상남도 진주시,None
17153,대구광역시 중구청,대구광역시 중구 동덕로36길 39,대구광역시 중구 동인동3가 3-1,생활방범,1,200,360도 전방면,30,2015-12,053-661-2494,35.870177,128.606579,2018-06-30,3410000,대구광역시 중구,"(동덕로36길, 중구, 대구, 41267, 대한민국, (35.8693644, 128..."
25554,경상남도 창원시,경상남도 창원시 의창구 봉곡로115번길 27,경상남도 창원시 의창구 봉곡동 160-16,생활방범,2,200,양방향,30,2019-12,055-225-3111,35.246135,128.673676,2020-08-25,5670000,경상남도 창원시,"(봉곡로115번길, 의창구, 창원시, 경상남도, 51157, 대한민국, (35.24..."


동일 URL(user_agent)로 과다한 요청이 발생하면 Time Out 에러가 발생할 수 있음
https://operations.osmfoundation.org/policies/nominatim/

In [10]:
# 좌표(coords)가 없는 데이터 삭제
df_cctv2.dropna(subset=['lat2'],axis=0,inplace=True)
df_cctv2.dropna(subset=['lon2'],axis=0,inplace=True)

In [18]:
df_cctv2

,admin_nm,addr_rd,addr_jb,purpose,cctv_cnt,pixel,direction,period,date_install,tel,...,lon,date_make,offer_admin_cd,offer_admin_nm,coords,location,Lat,Lon,lat2,lon2
15745,대구광역시 중구청,대구광역시 중구 남산로7길 75,대구광역시 중구 남산동 2482-458,어린이보호,1,200,360도 전방면,30,2016-05,053-661-2494,...,128.579142,2018-06-30,3410000,대구광역시 중구,"(남산로7길, 까치아파트, 남산동, 중구, 대구, 41977, 대한민국, (35.8...","(남산로7길, 까치아파트, 남산동, 중구, 대구, 41977, 대한민국, (35.8...",35.860233,128.579071,35.860233,128.579071
34151,서울특별시 동대문구,서울특별시 동대문구 천호대로85길 56,서울특별시 동대문구 장안1동 447-7,생활방범,3,200,NaN,NaN,2007-01,02-2127-4097,...,127.067368,2020-10-23,3050000,서울특별시 동대문구,"(천호대로85길, Jangan 1(il)-dong, 동대문구, 서울, 노원구, 02...","(천호대로85길, Jangan 1(il)-dong, 동대문구, 서울, 노원구, 02...",37.564207,127.067344,37.564207,127.067344
3492,경상남도 진주시청,경상남도 진주시 충무공동 바람모아생활방범 전망데크,경상남도 진주시 충무공동 바람모아생활방범 전망데크,생활방범,4,200,전방 50미터,30,2018-12,055-749-5566,...,128.132953,2019-10-23,5310000,경상남도 진주시,None,None,NaN,NaN,NaN,NaN
17153,대구광역시 중구청,대구광역시 중구 동덕로36길 39,대구광역시 중구 동인동3가 3-1,생활방범,1,200,360도 전방면,30,2015-12,053-661-2494,...,128.606579,2018-06-30,3410000,대구광역시 중구,"(동덕로36길, 중구, 대구, 41267, 대한민국, (35.8693644, 128...","(동덕로36길, 중구, 대구, 41267, 대한민국, (35.8693644, 128...",35.869364,128.609720,35.869364,128.609720
25554,경상남도 창원시,경상남도 창원시 의창구 봉곡로115번길 27,경상남도 창원시 의창구 봉곡동 160-16,생활방범,2,200,양방향,30,2019-12,055-225-3111,...,128.673676,2020-08-25,5670000,경상남도 창원시,"(봉곡로115번길, 의창구, 창원시, 경상남도, 51157, 대한민국, (35.24...","(봉곡로115번길, 의창구, 창원시, 경상남도, 51157, 대한민국, (35.24...",35.246408,128.672784,35.246408,128.672784
23035,경기도 이천시청,경기도 이천시 경충대로2598번길 32,경기도 이천시 중리동 459,어린이보호,1,200,120,30,2018-05,031-644-2942,...,127.444763,2020-01-20,4070000,경기도 이천시,None,None,NaN,NaN,NaN,NaN
36554,부산광역시 북구청,부산광역시 북구 만덕3로48번나길 17,부산광역시 북구 만덕동 913-1,생활방범,1,200,360도 전방면,30,2011-04,051-309-2031,...,129.027848,2020-03-20,3320000,부산광역시 북구,"(만덕3로48번나길, 만덕3동, 만덕동, 북구, 부산, 46571, 대한민국, (3...","(만덕3로48번나길, 만덕3동, 만덕동, 북구, 부산, 46571, 대한민국, (3...",35.211231,129.027707,35.211231,129.027707
28189,경상남도 거제시청,경상남도 거제시 거제중앙로 1553,경상남도 거제시 문동동 598-8,어린이보호,2,200,전방 100m,30,2018-08,055-639-3681,...,128.643336,2020-02-27,5370000,경상남도 거제시,"(1553, 거제중앙로, 상문동, 거제시, 경상남도, 53294, 대한민국, (34...","(1553, 거제중앙로, 상문동, 거제시, 경상남도, 53294, 대한민국, (34...",34.861679,128.643329,34.861679,128.643329
24872,체육시설사업소,경상북도 칠곡군 왜관읍 호국로 203 (아곡리 670-5),경상북도 칠곡군 왜관읍 호국로 203 (아곡리 670-5),시설물관리,1,41,NaN,30,2013-01,054-979-6881,...,128.423639,2018-08-22,5220000,경상북도 칠곡군,None,None,NaN,NaN,NaN,NaN
33268,경기도 동두천시청,경기도 동두천시 못골로49번길 107,경기도 동두천시 생연동 282-1,생활방범,5,200,NaN,30,2020-06,031-860-2834,...,127.063385,2020-09-18,3920000,경기도 동두천시,None,None,NaN,NaN,NaN,NaN


### 원 좌표와 네이버에서 구한 좌표와의 거리 차를 검토

In [11]:
# 거리를 계산하기 위한 라이브러리 로드
from geopy.distance import great_circle

In [12]:
# Haversine식을 이용한 거리 계산
##great_circle( (출발지 위도, 출발지 경도), (도착지 위도, 도착지 경도) )
df_cctv2['dist'] = df_cctv2.apply(lambda row: great_circle( (row['lat2'], row['lon2']), (row['lat'], row['lon'])).meters, axis=1)    
df_cctv2.head()

,admin_nm,addr_rd,addr_jb,purpose,cctv_cnt,pixel,direction,period,date_install,tel,lat,lon,date_make,offer_admin_cd,offer_admin_nm,location,lat2,lon2,dist
40686,강원도 원주시청,소방서길 22-5,명륜2동 820-6,생활방범,1,200,360,30,NaN,033-737-5153,37.329761,127.944557,2020-09-20,4190000,강원도 원주시,"(소방서길, 명륜동, 원주시, 강원도, 26481, 대한민국, (37.3316091...",37.331609,127.944570,205.447154
37776,공주시청,충청남도 공주시 금벽로 368,충청남도 공주시 신관동 553천(금강신관공원공주대교 아래),생활방범,1,130,360도전방면,30,2016-12,041-840-8500,36.462876,127.127437,2020-03-25,4500000,충청남도 공주시,"(금벽로, 노은1동, 노은동, 온천리, 공주시, 충청남도, 34158, 대한민국, ...",36.363186,127.268690,16812.226790
26562,부산광역시 금정구청,부산광역시 금정구 동부곡로15번길 3,부산광역시 금정구 부곡동 287-6,생활방범,1,200,360도 전방면,30,2020-05,051-519-5731,35.225864,129.093181,2020-08-12,3350000,부산광역시 금정구,"(동부곡로15번길, 부곡2동, 부곡동, 금정구, 부산, 46271, 대한민국, (3...",35.228317,129.093780,278.154903
19917,경상남도 창원시,경상남도 창원시 의창구 창이대로397번길 2,경상남도 창원시 의창구 사림동 75-1,교통정보수집,4,130,360도 전방면,30,2011-06,055-225-3111,35.242717,128.680032,2020-08-25,5670000,경상남도 창원시,"(창이대로397번길, 의창구, 창원시, 경상남도, 51431, 대한민국, (35.2...",35.244254,128.680770,183.549756
31971,전라북도 군산시,전라북도 군산시 번영로 2,전라북도 군산시 경장동 472,생활방범,2,200,360도 전방면,30,2017-12,063-454-7922,35.975177,126.723794,2020-03-12,4670000,전라북도 군산시,"(번영로, 대야면, 군산시, 전라북도, 54060, 대한민국, (35.9413032...",35.941303,126.831394,10391.407279


# 상용 서비스를 활용

In [13]:
import urllib
import json

In [14]:
# 네이버 API 정보 읽기 
f = open('../../etc/naver.txt', 'r')
lines = f.readlines()

api_id = lines[0].replace("\n", "")
api_key = lines[1].replace("\n", "")

In [15]:
# 네이버 Map API https://console.ncloud.com/mc/solution/naverService/application
def search_map(search_text):
    client_id = 'id'
    client_secret = 'secret'
    encText = urllib.parse.quote(str(search_text))
    url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query='+encText
    request = urllib.request.Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', api_id) 
    request.add_header('X-NCP-APIGW-API-KEY', api_key) 
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        print(response_body.decode('utf-8'))
        return response_body.decode('utf-8')
    else:
        print("Error Code:" + rescode)

In [16]:
# 단 건 테스트
x = []
y = []

temp_map = search_map('경기 부천시  소사본1동 181-26 2층 201호')  #NAVER search_map 함수 실행
temp_map = json.loads(temp_map)
try:
    #print(temp_map)
    temp_map = temp_map['addresses'][0]
    x = temp_map['x']
    y = temp_map['y']
except IndexError:
    pass

print(x, y)

{"status":"OK","meta":{"totalCount":1,"page":1,"count":1},"addresses":[{"roadAddress":"","jibunAddress":"경기도 부천시 소사본동 181-26","englishAddress":"181-26, Sosabon-dong, Bucheon-si, Gyeonggi-do, Republic of Korea","addressElements":[{"types":["SIDO"],"longName":"경기도","shortName":"경기도","code":""},{"types":["SIGUGUN"],"longName":"부천시","shortName":"부천시","code":""},{"types":["DONGMYUN"],"longName":"소사본동","shortName":"소사본동","code":""},{"types":["RI"],"longName":"","shortName":"","code":""},{"types":["ROAD_NAME"],"longName":"","shortName":"","code":""},{"types":["BUILDING_NUMBER"],"longName":"","shortName":"","code":""},{"types":["BUILDING_NAME"],"longName":"","shortName":"","code":""},{"types":["LAND_NUMBER"],"longName":"181-26","shortName":"181-26","code":""},{"types":["POSTAL_CODE"],"longName":"","shortName":"","code":""}],"x":"126.7953156","y":"37.4748725","distance":0.0}],"errorMessage":""}
126.7953156 37.4748725


In [17]:
x1 = []
y1 = []
length_df = len(df_cctv2)
print(length_df)

15


In [18]:
# 데이터프레임 행수만큼 읽으면서 addr 항목의 주소를 Naver API로 돌려서 좌표값을 받아 리스트에 추가  
for i in range(length_df):
    addr = df_cctv2.iloc[i]['addr_rd']
    #print(addr)
    #print(i)
    temp_map = search_map(addr)  #NAVER search_map 함수 실행
    temp_map = json.loads(temp_map)
    try:
        #print(temp_map)
        temp_map = temp_map['addresses'][0]
        x = temp_map['x']
        y = temp_map['y']
        x1.append(x)
        y1.append(y)    
    except:
        x1.append(0)
        y1.append(0)    

{"status":"OK","meta":{"totalCount":1,"page":1,"count":1},"addresses":[{"roadAddress":"강원도 원주시 소방서길 22-5","jibunAddress":"강원도 원주시 명륜동 820-6","englishAddress":"22-5, Sobangseo-gil, Wonju-si, Gangwon-do, Republic of Korea","addressElements":[{"types":["SIDO"],"longName":"강원도","shortName":"강원도","code":""},{"types":["SIGUGUN"],"longName":"원주시","shortName":"원주시","code":""},{"types":["DONGMYUN"],"longName":"명륜동","shortName":"명륜동","code":""},{"types":["RI"],"longName":"","shortName":"","code":""},{"types":["ROAD_NAME"],"longName":"소방서길","shortName":"소방서길","code":""},{"types":["BUILDING_NUMBER"],"longName":"22-5","shortName":"22-5","code":""},{"types":["BUILDING_NAME"],"longName":"","shortName":"","code":""},{"types":["LAND_NUMBER"],"longName":"820-6","shortName":"820-6","code":""},{"types":["POSTAL_CODE"],"longName":"26481","shortName":"26481","code":""}],"x":"127.9444656","y":"37.3297418","distance":0.0}],"errorMessage":""}
{"status":"OK","meta":{"totalCount":1,"page":1,"count":1},"addresses

{"status":"OK","meta":{"totalCount":1,"page":1,"count":1},"addresses":[{"roadAddress":"경상남도 하동군 화개면 쌍계로 15 화개장터","jibunAddress":"경상남도 하동군 화개면 탑리 726-46 화개장터","englishAddress":"15, Ssanggye-ro, Hwagae-myeon, Hadong-gun, Gyeongsangnam-do, Republic of Korea","addressElements":[{"types":["SIDO"],"longName":"경상남도","shortName":"경상남도","code":""},{"types":["SIGUGUN"],"longName":"하동군","shortName":"하동군","code":""},{"types":["DONGMYUN"],"longName":"화개면","shortName":"화개면","code":""},{"types":["RI"],"longName":"탑리","shortName":"탑리","code":""},{"types":["ROAD_NAME"],"longName":"쌍계로","shortName":"쌍계로","code":""},{"types":["BUILDING_NUMBER"],"longName":"15","shortName":"15","code":""},{"types":["BUILDING_NAME"],"longName":"화개장터","shortName":"화개장터","code":""},{"types":["LAND_NUMBER"],"longName":"726-46","shortName":"726-46","code":""},{"types":["POSTAL_CODE"],"longName":"52303","shortName":"52303","code":""}],"x":"127.6240669","y":"35.1879895","distance":0.0}],"errorMessage":""}
{"status":"OK","meta":{

In [19]:
# 좌표 리스트 값을 데이터프레임의 X, Y 항목에 넣어줌 
df_cctv2['x'] = x1
df_cctv2['y'] = y1

In [20]:
# 잘못 입력된 좌표 리턴값 "[]"을 치환
df_cctv2['x'] = df_cctv2['x'].str.replace('[', '0')
df_cctv2['x'] = df_cctv2['x'].str.replace(']', '0')
df_cctv2['y'] = df_cctv2['y'].str.replace('[', '0')
df_cctv2['y'] = df_cctv2['y'].str.replace(']', '0')
df_cctv2.head()

<ipython-input-20-30e685e7b4a0>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df_cctv2['x'] = df_cctv2['x'].str.replace('[', '0')
<ipython-input-20-30e685e7b4a0>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df_cctv2['x'] = df_cctv2['x'].str.replace(']', '0')
<ipython-input-20-30e685e7b4a0>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df_cctv2['y'] = df_cctv2['y'].str.replace('[', '0')
<ipython-input-20-30e685e7b4a0>:5: FutureWarning: The default value of regex will change from True to False in a future version. I

,admin_nm,addr_rd,addr_jb,purpose,cctv_cnt,pixel,direction,period,date_install,tel,...,lon,date_make,offer_admin_cd,offer_admin_nm,location,lat2,lon2,dist,x,y
40686,강원도 원주시청,소방서길 22-5,명륜2동 820-6,생활방범,1,200,360,30,NaN,033-737-5153,...,127.944557,2020-09-20,4190000,강원도 원주시,"(소방서길, 명륜동, 원주시, 강원도, 26481, 대한민국, (37.3316091...",37.331609,127.944570,205.447154,127.9444656,37.3297418
37776,공주시청,충청남도 공주시 금벽로 368,충청남도 공주시 신관동 553천(금강신관공원공주대교 아래),생활방범,1,130,360도전방면,30,2016-12,041-840-8500,...,127.127437,2020-03-25,4500000,충청남도 공주시,"(금벽로, 노은1동, 노은동, 온천리, 공주시, 충청남도, 34158, 대한민국, ...",36.363186,127.268690,16812.226790,127.1274390,36.4702150
26562,부산광역시 금정구청,부산광역시 금정구 동부곡로15번길 3,부산광역시 금정구 부곡동 287-6,생활방범,1,200,360도 전방면,30,2020-05,051-519-5731,...,129.093181,2020-08-12,3350000,부산광역시 금정구,"(동부곡로15번길, 부곡2동, 부곡동, 금정구, 부산, 46271, 대한민국, (3...",35.228317,129.093780,278.154903,129.0931920,35.2258501
19917,경상남도 창원시,경상남도 창원시 의창구 창이대로397번길 2,경상남도 창원시 의창구 사림동 75-1,교통정보수집,4,130,360도 전방면,30,2011-06,055-225-3111,...,128.680032,2020-08-25,5670000,경상남도 창원시,"(창이대로397번길, 의창구, 창원시, 경상남도, 51431, 대한민국, (35.2...",35.244254,128.680770,183.549756,128.6802964,35.2430056
31971,전라북도 군산시,전라북도 군산시 번영로 2,전라북도 군산시 경장동 472,생활방범,2,200,360도 전방면,30,2017-12,063-454-7922,...,126.723794,2020-03-12,4670000,전라북도 군산시,"(번영로, 대야면, 군산시, 전라북도, 54060, 대한민국, (35.9413032...",35.941303,126.831394,10391.407279,126.7238333,35.9751429


In [21]:
# 좌표 항목을 실수 형식으로 변환 
df_cctv2['x'] = df_cctv2['x'].astype(float)
df_cctv2['y'] = df_cctv2['y'].astype(float)

In [72]:
df_cctv2

,admin_nm,addr_rd,addr_jb,purpose,cctv_cnt,pixel,direction,period,date_install,tel,lat,lon,date_make,offer_admin_cd,offer_admin_nm,x,y
31746,전라북도 군산시,전라북도 군산시 삼화1길 7,전라북도 군산시 문화동 921,생활방범,1,200,360도 전방면,30,2015-06,063-454-7922,35.974253,126.706394,2020-03-12,4670000,전라북도 군산시,126.706381,35.974254
8454,서울특별시 노원구청,서울특별시 노원구 화랑로51길 17 솔밭근린공원,서울특별시 노원구 공릉동 111-1 (솔밭근린공원 -1),시설물관리,1,200,회전,30,2015-09,02-2116-3952,37.622720,127.088171,2019-04-08,3100000,서울특별시 노원구,127.088554,37.623989
43019,경상북도 영덕군청,경상북도 영덕군 영덕읍 우곡리 609,경상북도 영덕군 영덕읍 우곡리 609,생활방범,1,200,단방향,30,2016-06,054-730-6300,36.408073,129.373656,2019-09-17,5180000,경상북도 영덕군,129.374198,36.407927
12573,울산광역시 동구청,울산광역시 동구 진성9길 2,울산광역시 동구 일산동 456-9,생활방범,1,200,대송로 10 방향,30,2017-08,052-209-3145,35.504506,129.428526,2020-10-30,3710000,울산광역시 동구,129.428529,35.504509
7419,서울특별시 마포구청,와우산로30길,창전동 3-183,방범(공원),1,200,360도전방면,30,2012-07,02-3153-8432,37.552997,126.930324,2020-05-20,3130000,서울특별시 마포구,NaN,NaN
25390,부산광역시 수영구청,부산광역시 수영구 장대골로27번길 29,부산광역시 수영구 광안동 735-11,생활방범,1,200,360도전방면,30,2015-11,051-610-4492,35.157912,129.110367,2019-12-20,3380000,부산광역시 수영구,129.110358,35.157911
41489,전라북도 전주시청,전라북도 전주시 완산구 안행2길 25,전라북도 전주시 완산구 삼천동1가 739,어린이보호,2,200,반경100m이내,30,2012-12,063-281-2072,35.802009,127.129149,2020-08-31,4640000,전라북도 전주시,127.128724,35.801831
11684,서울특별시 강북구청,서울특별시 강북구 삼양로139가길 43,서울특별시 강북구 우이동 73-23,생활방범,1,200,NaN,30,2012-05,02-901-7266,37.651230,127.012093,2020-05-12,3080000,서울특별시 강북구,127.012015,37.651108
24365,안전관리과,경상북도 칠곡군 기산면 행정리 1180-1,경상북도 칠곡군 기산면 행정리 1180-1,차량방범,1,200,NaN,30,2014-07,054-979-6881,35.969625,128.363002,2018-08-22,5220000,경상북도 칠곡군,128.363010,35.969626
29014,서울특별시 성동구청,서울특별시 성동구 자동차시장길 22,서울특별시 성동구 용답동 223-5,생활방범,3,200,360도전방면,30,2014-09,02-2286-5879,37.561981,127.057297,2020-12-04,3030000,서울특별시 성동구,127.056381,37.561185


In [22]:
df_cctv2.dropna(subset=['x'],axis=0,inplace=True)  #거리 비교를 위해 null 삭제
df_cctv2.dropna(subset=['y'],axis=0,inplace=True)  #거리 비교를 위해 null 삭제

### 원 좌표와 네이버에서 구한 좌표와의 거리 차를 검토

In [23]:
# Haversine식을 이용한 거리 계산
##great_circle( (출발지 위도, 출발지 경도), (도착지 위도, 도착지 경도) )
df_cctv2['dist2'] = df_cctv2.apply(lambda x: great_circle((x['lat'], x['lon']), (x['y'], x['x'])).meters, axis=1)

In [75]:
df_cctv2

,admin_nm,addr_rd,addr_jb,purpose,cctv_cnt,pixel,direction,period,date_install,tel,lat,lon,date_make,offer_admin_cd,offer_admin_nm,x,y,dist
31746,전라북도 군산시,전라북도 군산시 삼화1길 7,전라북도 군산시 문화동 921,생활방범,1,200,360도 전방면,30,2015-06,063-454-7922,35.974253,126.706394,2020-03-12,4670000,전라북도 군산시,126.706381,35.974254,1.208316
8454,서울특별시 노원구청,서울특별시 노원구 화랑로51길 17 솔밭근린공원,서울특별시 노원구 공릉동 111-1 (솔밭근린공원 -1),시설물관리,1,200,회전,30,2015-09,02-2116-3952,37.622720,127.088171,2019-04-08,3100000,서울특별시 노원구,127.088554,37.623989,145.004489
43019,경상북도 영덕군청,경상북도 영덕군 영덕읍 우곡리 609,경상북도 영덕군 영덕읍 우곡리 609,생활방범,1,200,단방향,30,2016-06,054-730-6300,36.408073,129.373656,2019-09-17,5180000,경상북도 영덕군,129.374198,36.407927,51.153852
12573,울산광역시 동구청,울산광역시 동구 진성9길 2,울산광역시 동구 일산동 456-9,생활방범,1,200,대송로 10 방향,30,2017-08,052-209-3145,35.504506,129.428526,2020-10-30,3710000,울산광역시 동구,129.428529,35.504509,0.437615
25390,부산광역시 수영구청,부산광역시 수영구 장대골로27번길 29,부산광역시 수영구 광안동 735-11,생활방범,1,200,360도전방면,30,2015-11,051-610-4492,35.157912,129.110367,2019-12-20,3380000,부산광역시 수영구,129.110358,35.157911,0.783797
41489,전라북도 전주시청,전라북도 전주시 완산구 안행2길 25,전라북도 전주시 완산구 삼천동1가 739,어린이보호,2,200,반경100m이내,30,2012-12,063-281-2072,35.802009,127.129149,2020-08-31,4640000,전라북도 전주시,127.128724,35.801831,43.109825
11684,서울특별시 강북구청,서울특별시 강북구 삼양로139가길 43,서울특별시 강북구 우이동 73-23,생활방범,1,200,NaN,30,2012-05,02-901-7266,37.651230,127.012093,2020-05-12,3080000,서울특별시 강북구,127.012015,37.651108,15.208931
24365,안전관리과,경상북도 칠곡군 기산면 행정리 1180-1,경상북도 칠곡군 기산면 행정리 1180-1,차량방범,1,200,NaN,30,2014-07,054-979-6881,35.969625,128.363002,2018-08-22,5220000,경상북도 칠곡군,128.363010,35.969626,0.696153
29014,서울특별시 성동구청,서울특별시 성동구 자동차시장길 22,서울특별시 성동구 용답동 223-5,생활방범,3,200,360도전방면,30,2014-09,02-2286-5879,37.561981,127.057297,2020-12-04,3030000,서울특별시 성동구,127.056381,37.561185,119.795268
43993,서울특별시 금천구,서울특별시 금천구 독산로44길 51,서울특별시 금천구 시흥4동 793-6,다목적,4,200,360도방향,30,NaN,02-2627-1903,37.465061,126.907546,2020-07-31,3170000,서울특별시 금천구,126.907544,37.461831,359.160173


In [24]:
df_cctv2['dist2'].min()

0.1521647412979709

In [25]:
df_cctv2['dist2'].max()

6280.385223068997

In [26]:
df_cctv2['dist2'].mean()

558.7255814188676

In [27]:
df_cctv2['dist2'].median()

8.390073370422144

포털지도와 구글맵 등을 이용하여 도로명주소, 지번주소, 원 좌표, 도로명주소 좌표 등을 비교

In [81]:
#다음 과정을 위해 데이터프레임을 csv 파일로 저장 
df_cctv2.to_csv("./data/cctv_coord_sample.csv", encoding='euc-kr')